This script creates a class for backtesting, including the trading strategy. 

# Backtesting class

In [ ]:
!pip install backtesting
from backtesting import Backtest
from backtesting.lib import SignalStrategy

from google.colab import drive
import pandas as pd
import numpy as np
drive.mount('/content/drive',force_remount=True)


     |████████████████████████████████| 175 kB 11.1 MB/s 
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173822 sha256=ca19b0fd223e369e4116e713f4bbd598600b23a225990d49d63d94cf5262305c
  Stored in directory: /root/.cache/pip/wheels/48/1d/ea/65dacebc37da7655d8a1fd0f315ac39d102e31d6545237a1c6
Successfully built backtesting


/usr/local/lib/python3.7/dist-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Mounted at /content/drive


# Wrap up a function and ready for calling from other script

In [ ]:
# Trading Strategy: Buy (Sell) when the state changes to 1 (-1). Close out all positions when the state changes. Do nothing when the state holds.
class TradingStrategy(SignalStrategy):
    def init(self):
        super().init()
    def next(self):
      if self.data.State[-2] != self.data.State[-1]:
        if self.position:
          self.position.close()
        if self.data.State[-1] == 1:
          self.buy(0.13)
        elif self.data.State[-1] == -1:
          self.sell()

def backtesting(df_px_label, cash = 1000000, commission=0.0025, TradingStrategy = TradingStrategy, plotGraph=True):
  ###  df_px_label is a dataframe with column 'State' and 'Close' ###
  if ~np.isin(['State','Close'], df_px_label.columns).all():
    ValueError("backtesting: Dataframe input must contain columns with name 'State' and 'Close' ")
  df_px_label = df_px_label.copy()
  for c in ['Open','High','Low']:
    if c not in df_px_label.columns:
      df_px_label[c] = df_px_label.Close
  print(df_px_label.head())

  bt = Backtest(df_px_label,  TradingStrategy,cash=cash, commission=commission,trade_on_close=True)
  stats = bt.run()
  if plotGraph:
    bt.plot()
    print(stats)
  return bt, stats, df_px_label

def backtesting_df(df,close_name,label_name, cash = 1000000, commission=0.0025, TradingStrategy = TradingStrategy, plotGraph=True):
  df_px_label = df[[close_name,label_name]]
  df_px_label.columns = ['Close','State']
  return backtesting(df_px_label, cash , commission, TradingStrategy, plotGraph )


# Example

In [ ]:
# import os
# DATA_DIR="/content/drive/MyDrive/IAQF - Five+1 Guys/2022/Data"
# LABEL_DIR = os.path.join(DATA_DIR,'Label')
# df_dir = os.path.join(LABEL_DIR,'master_ds_v1.csv')
# data = pd.read_csv(df_dir, parse_dates=True, index_col=0)
# data
# data = pd.read_excel('/content/drive/MyDrive/IAQF - Five+1 Guys/2022/Data/Backtesting_TradingStrategy/TradingStrategyInputSample.xlsx', index_col='Date')

In [ ]:
# #data = data[['Russell_Close','POS_RET']]
# #data.columns = ['Close','State']
# data=data[['Close','State']]
# data

,Close,State
Date,,
2010-01-04,664.049988,-1.0
2010-01-05,665.969971,0.0
2010-01-06,666.549988,1.0
2010-01-07,669.340027,0.0
2010-01-08,671.489990,1.0
...,...,...
2021-12-01,2714.169920,0.0
2021-12-02,2660.810060,0.0
2021-12-03,2633.860110,0.0


In [ ]:
# data.index

DatetimeIndex(['2010-01-04', '2010-01-05', '2010-01-06', '2010-01-07',
               '2010-01-08', '2010-01-11', '2010-01-12', '2010-01-13',
               '2010-01-14', '2010-01-15',
               ...
               '2021-11-23', '2021-11-24', '2021-11-26', '2021-11-29',
               '2021-11-30', '2021-12-01', '2021-12-02', '2021-12-03',
               '2021-12-06', '2021-12-07'],
              dtype='datetime64[ns]', name='Date', length=3006, freq=None)

In [ ]:
# bt, stats = backtesting(data, commission=0.0025)

                 Close  State        Open        High         Low
Date                                                             
2010-01-04  664.049988   -1.0  664.049988  664.049988  664.049988
2010-01-05  665.969971    0.0  665.969971  665.969971  665.969971
2010-01-06  666.549988    1.0  666.549988  666.549988  666.549988
2010-01-07  669.340027    0.0  669.340027  669.340027  669.340027
2010-01-08  671.489990    1.0  671.489990  671.489990  671.489990


Start                     2010-01-04 00:00:00
End                       2021-12-07 00:00:00
Duration                   4355 days 00:00:00
Exposure Time [%]                    89.18829
Equity Final [$]                  6951.837235
Equity Peak [$]                  10069.361015
Return [%]                         -30.481628
Buy & Hold Return [%]              284.389743
Return (Ann.) [%]                   -3.001986
Volatility (Ann.) [%]                3.825415
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -30.960493
Avg. Drawdown [%]                   -3.544576
Max. Drawdown Duration     4246 days 00:00:00
Avg. Drawdown Duration      483 days 00:00:00
# Trades                                 1197
Win Rate [%]                        45.697577
Best Trade [%]                       8.695907
Worst Trade [%]                    -10.204901
Avg. Trade [%]                    